# Introduction

**Generator step by step**
1. Generate all random fonts map
2. Generate fonts based on maps
3. Replace all character inside document with new mapping
4. Inject new fonts into document
5. Change font and save document

## Database setup

In [1]:
import pymongo

mongo = pymongo.MongoClient('mongodb://localhost:27017/')
db = mongo['skripsi']
col = db['fonts']

# Generate all random fonts map

In [2]:
import string
import random

letters = list(string.ascii_lowercase)
random.shuffle(letters)

total_swaps = random.randint(10, 15)
random_letters = letters[:total_swaps]
letters = letters[:total_swaps]

random.shuffle(random_letters)
swaps = dict(zip(letters, random_letters))
swaps

{'u': 'k',
 'b': 'n',
 'e': 'y',
 'l': 'b',
 'k': 'o',
 'y': 'x',
 'o': 'u',
 'r': 't',
 'x': 'e',
 'n': 'r',
 't': 'f',
 'f': 'l'}

# Generate fonts based on maps

In [3]:
import uuid
from fontTools import ttLib
from copy import deepcopy

font_input_path = '../inputs/times.ttf'
font_output_path = '../tmps/' + uuid.uuid4().hex + '.ttf'

font = ttLib.TTFont(font_input_path)
original_font = deepcopy(font)

for key, value in swaps.items():
    font['glyf'][key] = original_font['glyf'][value]  # Swap glyph
    font['hmtx'].metrics[key] = original_font['hmtx'].metrics[value] # Swap metric

font.save(font_output_path)

In [4]:
import os

os.system(f"python ../tests/rename-fonts.py \"{font_output_path}\" -s \" Fake\" --inplace")

0

# Save to MongoDB

In [5]:
summary = {
    'swaps': swaps,
    'font': os.path.basename(font_output_path),
    'fontname': 'Times New Roman Fake',
}

col.insert_one(summary)
summary

{'swaps': {'u': 'k',
  'b': 'n',
  'e': 'y',
  'l': 'b',
  'k': 'o',
  'y': 'x',
  'o': 'u',
  'r': 't',
  'x': 'e',
  'n': 'r',
  't': 'f',
  'f': 'l'},
 'font': 'caf3c1265a8446beabde7cb9a260a0aa.ttf',
 '_id': ObjectId('6330eedf3f847584598c2b3a')}

# Custom fonts

In [17]:
def generate_pair(char1, char2):
    return (
        f"{char1} - {char2}",
        {char1:char2, char2:char1, char1.upper():char2.upper(), char2.upper():char1.upper()}
    )

In [18]:
import os
from fontTools import ttLib
from copy import deepcopy

data = [
    generate_pair("a", "d"),
    generate_pair("i", "e"),
    generate_pair("u", "f"),
    generate_pair("e", "g"),
]

for index, (title, swaps) in enumerate(data):
    index += 1

    font_input_path = '../inputs/FakeTNR.ttf'
    font_output_path = f'../tmps/font {index} {title}.ttf'

    font = ttLib.TTFont(font_input_path)
    original_font = deepcopy(font)

    for key, value in swaps.items():
        font['glyf'][key] = original_font['glyf'][value]  # Swap glyph
        font['hmtx'].metrics[key] = original_font['hmtx'].metrics[value] # Swap metric

    font.save(font_output_path)

    os.system(f"python ../tests/rename_fonts.py \"{font_output_path}\" -s \" Font {index}\" --inplace")

# Swap by glyph order

In [24]:
from fontTools import ttLib
from copy import deepcopy

font_input_path = '../inputs/times.ttf'
font_output_path = '../tmps/00000 font.ttf'

font = ttLib.TTFont(font_input_path)

glyph_order = font.getGlyphOrder()
original_glyph_order = deepcopy(glyph_order)

a_index = glyph_order.index('a')
x_index = glyph_order.index('x')

glyph_order[a_index], glyph_order[x_index] = glyph_order[x_index], glyph_order[a_index]

font.setGlyphOrder(glyph_order)
font.save(font_output_path)

In [26]:
original_glyph_order == glyph_order

False